In [1]:
%matplotlib inline
path = "/home/ubuntu/data2/dogscats/"
home = "/home/ubuntu/"
#set home directory
import os
os.chdir(home)

import utils; reload(utils)
from utils import *
print ("loaded.")

loaded.


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
batch_size = 64

To do list- from kaggle comp instructions
https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition
1. create validation and sample
2. move to separate dirs for each set
3. finetune and train
4. submit

In [3]:
%cd data2/dogscats
# splitting of files from test into validation already done.
# splitting of files into categories dogs and cats already done in valid directory.

/home/ubuntu/data2/dogscats


In [4]:
from vgg16 import Vgg16

In [5]:
vgg = Vgg16()

In [6]:
print("started")
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
vgg.finetune(batches)
print ("done")

started
Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
done


In [7]:
print("start")
vgg.fit(batches, val_batches, nb_epoch=1)
vgg.model.save_weights('dogs-cats-results_1epoch_1.h5')
print("end__")

start
Epoch 1/1
23000/23000 [==============================] - 719s - loss: 0.1244 - acc: 0.9673 - val_loss: 0.0552 - val_acc: 0.9820
end


In [8]:
vgg.model.optimizer.lr = 0.01 #refer lesson 2 video @ 38:05, might improve results.

In [9]:
#this gave Exception: output of generator should be a tuple (x, y, sample_weight) or (x, y). 
#refer lesson 2 video @ 38.09
print("start")
vgg.fit(batches, val_batches, nb_epoch=1)
vgg.model.save_weights('results_ft1_5epoch_2nd_run.h5')
print("end")
#https://keras.io/models/about-keras-models/
#model.load_weights(filepath, by_name=False)

start
Epoch 1/1
23000/23000 [==============================] - 722s - loss: 0.0999 - acc: 0.9757 - val_loss: 0.0694 - val_acc: 0.9845
end


In [ ]:
#in command prompt. move files from test1 to test1/a
#vgg.test needs files to be in one directory otherwise fails w error message zero files found & /0 error
print ("vgg.test start")
batches, preds = vgg.test(path+'test1', batch_size = batch_size*2)
print("type(batches):", type(batches))
print("type(batches.filenames):", type(batches.filenames))
print("type(preds):", type(preds))


In [ ]:
#store results in file so we don't lose the results from long detailed calc.

filenames = batches.filenames
save_array(path+'test_preds.dat', preds)
save_array(path+'filenames.dat', filenames)


In [ ]:
len(batches.filenames)

In [ ]:
batches.filenames[0:5]

In [ ]:
preds[0:5]
#NB: in lecture the output is 1. and 0. not a range of non-int probs summing to 1.0

In [ ]:
preds = load_array('test_preds.dat')
filenames = load_array('filenames.dat')

In [ ]:
print(type(preds), preds.shape)
print(type(filenames), filenames.shape)
#we can see 12500 rows of data

https://keras.io/models/sequential/  
predict_generator(self, generator, steps, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)  
also refer vgg16.py  
def test(self, path, batch_size=8)  
return test_batches, self.model.predict_generator(test_batches, test_batches.nb_sample)  
NB: class_mode=None  
def get_batches(self, path, gen=image.ImageDataGenerator(), shuffle=True, batch_size=8, class_mode='categorical'):  
class_mode='categorical'  




In [ ]:
from PIL import Image
#nb: jupyter notebook only displays the last output.
#Image.open('test1/'+filenames[0])
Image.open('test1/'+filenames[1])

In [ ]:
isdog = preds[:, 1]
isdog[0:5]

In [ ]:
isdog = isdog.clip(0.05, 0.95)
isdog[0:5]

In [ ]:
#strip out the int's from filename and store in list.
#ids = [f for f in filenames]
#print(ids[:5])
ids = [int(f[2:f.find('.')]) for f in filenames]
ids[:5]

In [ ]:
subm = np.stack([ids, isdog], axis=1)
subm[:5]

In [ ]:
np.savetxt('submit.csv', subm, fmt='%d,%.5f', header='id,label', comments='')

In [ ]:
os.getcwd()

In [ ]:
from Ipython.display import FileLink
FileLink('submit.csv')
#error. library not installed? tried pip install

In [ ]:
print("calc confusion matrix")
cm = confusion_matrix(labels, preds)
print("confusion matrix created.", type(cm))
plot_confusion_matrix(cm, val_batches.class_indices)